<a href="https://colab.research.google.com/github/mqsRafa/mantenha-se-atualizado/blob/main/Mantenha_se_atualizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# -*- coding: utf-8 -*-
"""Notebook Colab: Sistema Multiagente de Notícias

Este notebook implementa um sistema multiagente utilizando a biblioteca Google ADK e o modelo Gemini
para buscar, curar e resumir notícias relevantes com base na localização do usuário.
"""

# @title Instalação das bibliotecas necessárias
# Instala as bibliotecas google-genai e google-adk
%pip -q install google-genai google-adk

# @title Configuração da API Key do Google Gemini
import os
from google.colab import userdata

# Configura a variável de ambiente com a API Key.
# Certifique-se de que a sua API Key está armazenada nos Secrets do Colab com o nome 'GOOGLE_API_KEY'.
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("API Key configurada com sucesso.")
except userdata.SecretNotFoundError:
    print("Erro: Chave 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    print("Por favor, adicione sua API Key do Google Gemini nos Secrets do Colab.")
except Exception as e:
    print(f"Ocorreu um erro ao configurar a API Key: {e}")


# @title Configuração do cliente da SDK do Gemini e modelo
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search # Ferramenta para buscar no Google
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Pode ser útil para futuras implementações de busca de conteúdo de URL (não usado diretamente neste rascunho)
import warnings

# Ignora avisos para uma saída mais limpa no Colab
warnings.filterwarnings("ignore")

# Configura o cliente Gemini
try:
    client = genai.Client()
    MODEL_ID = "gemini-2.0-flash" # Modelo Gemini a ser utilizado
    print(f"Cliente Gemini configurado com o modelo: {MODEL_ID}")
except Exception as e:
    print(f"Erro ao configurar o cliente Gemini: {e}")


# @title Funções Auxiliares

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    """
    Envia uma mensagem para um agente e retorna a resposta final.

    Args:
        agent: O objeto Agent a ser chamado.
        message_text: O texto da mensagem de entrada para o agente.

    Returns:
        A resposta final do agente como uma string.
    """
    # Cria um serviço de sessão em memória para gerenciar o estado da conversa
    session_service = InMemorySessionService()
    # Cria uma nova sessão (pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente, que gerencia a execução do agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada no formato esperado pelo Gemini
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    # Captura apenas a resposta final
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response.strip() # Remove espaços em branco no início e fim

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  """
  Formata texto para exibição em Markdown no Google Colab.

  Args:
      text: O texto a ser formatado.

  Returns:
      Um objeto IPython.display.Markdown.
  """
  text = text.replace('•', '  *') # Substitui marcadores de lista comuns por marcadores Markdown
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True)) # Adiciona "> " no início de cada linha para formatar como citação


# @title Definição dos Agentes

##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
# Este agente busca notícias recentes (última semana) para uma cidade, país e globalmente.
def agente_buscador_noticias(cidade: str, pais: str) -> str:
    """
    Agente que busca links de notícias recentes para uma cidade, país e globalmente.

    Args:
        cidade: Nome da cidade fornecida pelo usuário.
        pais: Nome do país onde a cidade se localiza.

    Returns:
        Uma string contendo os links das notícias encontradas, organizados por categoria.
    """
    buscador = Agent(
        name="agente_buscador_noticias",
        model=MODEL_ID,
        instruction=f"""
        Você é um assistente de pesquisa de notícias. Sua tarefa é usar a ferramenta de busca do Google (google_search)
        para encontrar links de notícias publicadas na última semana sobre eventos e acontecimentos relevantes.
        Você deve realizar três buscas:
        1. Notícias relevantes na última semana para a cidade de {cidade}.
        2. Notícias relevantes na última semana para o país {pais}.
        3. Notícias globais relevantes na última semana.

        Para cada busca, liste os links das notícias encontradas. Se não encontrar notícias para uma categoria, indique isso.
        Formate a saída claramente, separando os links por categoria (Cidade, País, Global).
        Priorize links de fontes de notícias confiáveis.
        """,
        description="Agente que busca links de notícias recentes por localização e globalmente.",
        tools=[google_search] # Habilita a ferramenta de busca do Google
    )

    # Mensagem de entrada para o agente buscador
    entrada_do_agente_buscador = f"Por favor, encontre notícias recentes para a cidade de {cidade}, para o país {pais} e notícias globais relevantes da última semana."

    print("Agente 1 (Buscador de Notícias) em execução...")
    # Executa o agente e obtém a resposta contendo os links
    links_noticias = call_agent(buscador, entrada_do_agente_buscador)
    print("Agente 1 concluído.")
    return links_noticias

##################################################
# --- Agente 2: Especialista em Jornalismo --- ---#
##################################################
# Este agente recebe os links e filtra as notícias mais relevantes e urgentes.
# Nota: A "urgência" e "relevância" serão inferidas pelo modelo com base nas informações disponíveis nos snippets
# ou no que o google_search puder retornar sobre os links. Acessar o conteúdo completo das URLs exigiria outra ferramenta.
def agente_jornalista_curador(links_noticias: str) -> str:
    """
    Agente que filtra as notícias mais relevantes a partir de uma lista de links.

    Args:
        links_noticias: Uma string contendo os links de notícias fornecidos pelo Agente 1.

    Returns:
        Uma string contendo as notícias (ou resumos/títulos) das notícias mais relevantes.
    """
    curador = Agent(
        name="agente_jornalista_curador",
        model=MODEL_ID,
        instruction=f"""
        Você é um especialista em jornalismo e curadoria de notícias. Sua tarefa é analisar a lista de links de notícias fornecida
        e identificar as notícias mais relevantes e urgentes da última semana.
        Considere a importância do tema, a cobertura por diferentes fontes e o potencial impacto.
        Você não tem acesso direto ao conteúdo completo dos links, então baseie sua análise nos títulos e snippets retornados pela busca,
        e utilize a ferramenta google_search para obter mais contexto sobre os temas, se necessário.
        Selecione 10 notícias que você considera as mais importantes. Se não houverem ao menos 10 notícias, utilize todas as notícias encontradas.
        Para cada notícia selecionada, forneça o título e o link original.
        Se não houver notícias relevantes ou urgentes na lista, indique isso.
        """,
        description="Agente que filtra e seleciona as notícias mais relevantes e urgentes.",
        tools=[google_search] # Pode usar google_search para obter mais contexto sobre os temas
    )

    # Mensagem de entrada para o agente curador
    entrada_do_agente_curador = f"Analise esta lista de links de notícias e selecione as 10 mais relevantes e urgentes:\n\n{links_noticias}"

    print("Agente 2 (Especialista em Jornalismo) em execução...")
    # Executa o agente e obtém a resposta com as notícias filtradas
    noticias_filtradas = call_agent(curador, entrada_do_agente_curador)
    print("Agente 2 concluído.")
    return noticias_filtradas

##############################################
# --- Agente 3: Explicador Simples --- --- ---#
##############################################
# Este agente recebe as notícias filtradas e as explica de forma simples.
def agente_explicador_simples(noticias_filtradas: str) -> str:
    """
    Agente que explica notícias de forma simples e clara.

    Args:
        noticias_filtradas: Uma string contendo as notícias selecionadas pelo Agente 2.

    Returns:
        Uma string com as notícias explicadas de forma simplificada.
    """
    explicador = Agent(
        name="agente_explicador_fofoca",
        model=MODEL_ID,
        instruction=f"""
        Você é um comunicador especializado em explicar informações complexas de forma simples e clara.
        Recebeu uma lista das notícias mais relevantes. Sua tarefa é explicar o conteúdo dessas notícias como se fosse uma fofoca, utilizando linguagem acessível.
        Apresente cada notícia selecionada de forma concisa e fácil de entender, mas não simplifique muito. Dê detalhes importantes, afinal, é uma fofoca.
        Se a entrada indicar que não há notícias relevantes, apenas confirme isso.
        Utilize emojis para enriquecer a experiência do leitor.
        """,
        description="Agente que explica notícias de forma simples.",
        tools=[] # Não precisa de ferramentas externas
    )

    # Mensagem de entrada para o agente explicador
    entrada_do_agente_explicador = f"Por favor, explique as seguintes notícias de forma que soe como uma fofoca. Dê detalhes importantes:\n\n{noticias_filtradas}"

    print("Agente 3 (Explicador Fofoca) em execução...")
    # Executa o agente e obtém a explicação simplificada
    explicacao_noticias = call_agent(explicador, entrada_do_agente_explicador)
    print("Agente 3 concluído.")
    return explicacao_noticias

# @title Fluxo Principal de Execução

# Obtém a cidade do usuário
cidade_usuario = input("❓ Por favor, digite o nome da sua cidade: ")

# TODO: Implementar lógica para obter o país a partir da cidade (pode usar google_search ou outra API)
# Por enquanto, vamos pedir o país ao usuário ou definir um padrão.
# Para simplificar neste rascunho, vamos pedir o país ao usuário.
pais_usuario = input(f"❓ Por favor, digite o nome do país onde {cidade_usuario} se localiza: ")


if not cidade_usuario or not pais_usuario:
    print("Você esqueceu de digitar a cidade ou o país!")
else:
    print(f"\n🚀 Iniciando a busca e resumo de notícias para {cidade_usuario}, {pais_usuario}...")

    # --- Passo 1: Buscar Notícias ---
    print("\n--- Executando Agente 1: Buscador de Notícias ---")
    links_encontrados = agente_buscador_noticias(cidade_usuario, pais_usuario)
    print("\n--- Resultado do Agente 1 ---")
    display(to_markdown(links_encontrados))
    print("--------------------------------------------------------------")

    # --- Passo 2: Curar Notícias ---
    if "não encontrar notícias" not in links_encontrados.lower() and links_encontrados.strip():
        print("\n--- Executando Agente 2: Especialista em Jornalismo ---")
        noticias_curadas = agente_jornalista_curador(links_encontrados)
        print("\n--- Resultado do Agente 2 ---")
        display(to_markdown(noticias_curadas))
        print("--------------------------------------------------------------")

        # --- Passo 3: Explicar Notícias ---
        if "não houver notícias relevantes" not in noticias_curadas.lower() and noticias_curadas.strip():
            print("\n--- Executando Agente 3: Explicador Simples ---")
            explicacao_final = agente_explicador_simples(noticias_curadas)
            print("\n--- Resultado do Agente 3 ---")
            display(to_markdown(explicacao_final))
            print("--------------------------------------------------------------")
        else:
            print("\nNão foram encontradas notícias relevantes para explicar.")
    else:
        print("\nNão foram encontrados links de notícias pelo Agente 1.")

    print("\n✅ Processo concluído.")



API Key configurada com sucesso.
Cliente Gemini configurado com o modelo: gemini-2.0-flash
❓ Por favor, digite o nome da sua cidade: Jundiaí
❓ Por favor, digite o nome do país onde Jundiaí se localiza: Brasil

🚀 Iniciando a busca e resumo de notícias para Jundiaí, Brasil...

--- Executando Agente 1: Buscador de Notícias ---
Agente 1 (Buscador de Notícias) em execução...
Agente 1 concluído.

--- Resultado do Agente 1 ---


> Aqui estão alguns links de notícias recentes das últimas semanas, categorizados por localização:
> 
> ### Jundiaí
> 
> *   **Prefeitura de Jundiaí:**
>     *   Carreta da Mamografia inicia atendimentos em Jundiaí.
>     *   Vacina contra gripe é aplicada no grupo prioritário.
>     *   Fundo Social abre inscrição para curso de Power Platform – Microsoft (PL-900).
>     *   Visita de representantes da Prefeitura de Jundiaí à Flex Aero reforça compromisso com o desenvolvimento econômico e tecnológico da cidade.
>     *   Programa 'Quem Ama Cuida' inicia semana com ações em bairros das regiões Norte e Central.
>     *   Fundo Social de Solidariedade de Jundiaí (Funss), em parceria com o SENAC, oferecerá bolsa de estudos integral para curso de Assistente de Logística.
>     *   Prefeitura de Jundiaí iniciou os trabalhos de manutenção e conservação da Praça Mario Magaglio.
>     *   Mobilização em prol do Maio Laranja, mês de conscientização e combate à violência sexual contra crianças e adolescentes.
>     *   DAE Jundiaí está executando três obras de adutoras em diferentes regiões da cidade para garantir o abastecimento de água.
> *   **Jundiaí Online:** (notícias mais antigas, a partir de 2021)
> *   **Jornal da Cidade Jundiaí e Região:**
>     *   Com mais fiscalização, multas por dirigir bêbado aumentam 83,8%.
>     *   Jundiaí apura vazamento de corante que tingiu aves em parque.
>     *   Arborização reforça preservação ambiental na Vila Hortolândia.
>     *   FENS nos Bairros movimenta R$ 176 mil e atrai mais de 2 mil visitantes.
>     *   Praça José Godoy Ferraz, no Jardim Bonfiglioli, está revitalizada.
>     *   Jundiaí sedia o lançamento da 1ª Expo ESG.
> *   **Jornal da Região:**
>     *   Guarda de Jundiaí recupera dois tratores roubados.
>     *   Polícia Civil prende morador do Tamoio por Porte de Ilegal de Arma.
> *   **IA Notícias:**
>     *   Foragido por homicídio qualificado é preso no Ceasa de Jundiaí.
>     *   Torre de Segurança reforça vigilância e já mostra resultados em Jundiaí.
> 
> ### Brasil
> 
> *   **Brasil de Fato:**
>     *   Portabilidade de consignado para CLT entre bancos começa a valer.
>     *   Dilma participa de seminário online sobre papel do Banco do Brics.
>     *   Cracolândia: Tarcísio e Nunes estão aplicando 'estado de exceção seguindo a linha do fascismo', afirma sociólogo.
> *   **CNN Brasil:**
>     *   Gripe aviária: México, Chile e Uruguai suspendem compra de frango do Brasil.
>     *   Santa Catarina emite alerta máximo após gripe aviária em granja comercial no RS.
>     *   Agro brasileiro avança na China com novos acordos e escritório em Pequim.
> *   **Estadão:**
> *   **R7:**
>     *   Suspeito de contrabando é preso em MG com diamantes e US$ 124 mil.
>     *   Em protesto, familiares de feirante morto cobram condenação de PM.
>     *   Investigadores identificam quadrilha que matou policial penal em São Paulo.
> *   **Gazeta do Povo:**
>     *   Brasil suspende cooperação jurídica com o Peru em casos da Odebrecht na Lava Jato.
>     *   Argentina adere a embargo de aves do Brasil ao lado de China e UE.
> 
> ### Global
> 
> *   **CNN Brasil:**
>     *   Nigéria: Militantes matam ao menos 23 em ataque.
>     *   Trump diz que vai conversar com Putin e Zelensky na segunda-feira (19).
>     *   Líderes árabes sobem tom contra Israel em meio a escalada de bombardeios.
> *   **Estadão:**
>     *   Israel lança nova operação militar em Gaza para pressionar Hamas a libertar reféns.
>     *   Ataque russo mata 9 civis na Ucrânia horas após tentativa de acordo.
>     *   Medo da Rússia leva países europeus a comprarem sistemas de defesa aérea de Israel.
> *   **Terra:**
>     *   Bombardeios israelenses matam mais de cem em Gaza nas últimas 24 horas.
>     *   Zelensky está aberto a negociações no Vaticano, diz Kiev.
> *   **Gazeta do Povo:**
>     *   Trump torna público áudio que revela problemas de memória de Biden.
>     *   Nova ofensiva de Israel em Gaza deixa 146 mortos.
>     *   Moody's retira a nota máxima de crédito dos EUA após mais de um século.

--------------------------------------------------------------

--- Executando Agente 2: Especialista em Jornalismo ---
Agente 2 (Especialista em Jornalismo) em execução...
Agente 2 concluído.

--- Resultado do Agente 2 ---


> Aqui estão as 10 notícias mais relevantes e urgentes selecionadas a partir da lista fornecida:
> 
> 1.  **Santa Catarina emite alerta máximo após gripe aviária em granja comercial no RS** [CNN Brasil]. A detecção de gripe aviária em uma granja comercial tem implicações significativas para a saúde pública e o comércio internacional do Brasil, justificando o alerta máximo.
> 
> 2.  **Gripe aviária: México, Chile e Uruguai suspendem compra de frango do Brasil** [CNN Brasil]. Essa suspensão de compras por importantes parceiros comerciais devido à gripe aviária pode ter um impacto econômico considerável.
> 
> 3.  **Israel lança nova operação militar em Gaza para pressionar Hamas a libertar reféns** [Estadão]. O conflito Israel-Palestina continua sendo uma questão global crítica, e novas operações militares indicam uma escalada da violência.
> 
> 4.  **Bombardeios israelenses matam mais de cem em Gaza nas últimas 24 horas** [Terra]. O alto número de fatalidades em Gaza destaca a intensidade do conflito e suas consequências humanitárias.
> 
> 5.  **Nova ofensiva de Israel em Gaza deixa 146 mortos** [Gazeta do Povo]. Este evento ressalta a intensificação do conflito e suas graves implicações humanitárias, com um número elevado de vítimas.
> 
> 6.  **Ataque russo mata 9 civis na Ucrânia horas após tentativa de acordo** [Estadão]. O conflito na Ucrânia continua a gerar violência e perda de vidas civis, mesmo em meio a tentativas de acordo.
> 
> 7.  **Medo da Rússia leva países europeus a comprarem sistemas de defesa aérea de Israel** [Estadão]. Este fato demonstra como a tensão geopolítica com a Rússia está remodelando as estratégias de defesa na Europa.
> 
> 8.  **Trump diz que vai conversar com Putin e Zelensky na segunda-feira (19)** [CNN Brasil]. Uma possível conversa entre Trump, Putin e Zelensky pode indicar potenciais mudanças na diplomacia internacional.
> 
> 9.  **Moody's retira a nota máxima de crédito dos EUA após mais de um século** [Gazeta do Povo]. A alteração na avaliação de crédito dos EUA pela Moody's pode ter amplas implicações econômicas globais.
> 
> 10. **Brasil suspende cooperação jurídica com o Peru em casos da Odebrecht na Lava Jato** [Gazeta do Povo]. Essa suspensão pode afetar as investigações de corrupção em andamento e as relações bilaterais entre Brasil e Peru.
> 
> Para garantir que as informações acima estejam atualizadas e para obter mais contexto, especialmente sobre os eventos mais recentes como os conflitos em Gaza e Ucrânia, e as questões econômicas envolvendo o Brasil, vou realizar algumas pesquisas no Google.
> 
> Com base nas pesquisas adicionais, aqui estão algumas atualizações e insights sobre as notícias selecionadas:
> 
> *   **Gripe Aviária no Brasil:** O primeiro foco de gripe aviária em granja comercial no Brasil foi confirmado em Montenegro, Rio Grande do Sul. As autoridades já estão tomando medidas para conter a disseminação, incluindo o abate sanitário das aves. A China e a União Europeia já suspenderam a compra de carne de frango brasileira. A população pode ficar tranquila em relação ao consumo de carne de frango e ovos, pois a doença não é transmitida dessa forma.
> 
> *   **Conflito Israel-Gaza:** A situação em Gaza continua tensa, com relatos de ataques israelenses e um alto número de mortos. Há esforços em andamento para retomar as negociações de paz, mas a perspectiva de um cessar-fogo imediato é incerta.
> 
> *   **Guerra na Ucrânia:** Delegações da Rússia e da Ucrânia se reuniram na Turquia para discutir o fim da guerra, em negociações diretas pela primeira vez em mais de três anos. Houve um acordo para uma troca de prisioneiros em larga escala.
> 
> *   **Cooperação Jurídica Brasil-Peru:** A Secretaria Nacional de Justiça suspendeu a cooperação jurídica com o Peru em casos relacionados à Odebrecht na Lava Jato. A decisão foi motivada pelo uso de provas invalidadas pelo STF.
> 
> As notícias sobre a gripe aviária no Brasil e os conflitos internacionais (Israel-Gaza e Ucrânia) permanecem como os temas mais urgentes e relevantes devido às suas implicações econômicas, humanitárias e geopolíticas.

--------------------------------------------------------------

--- Executando Agente 3: Explicador Simples ---
Agente 3 (Explicador Fofoca) em execução...
Agente 3 concluído.

--- Resultado do Agente 3 ---


> E aí, pessoal, preparados para as últimas fofocas que estão BOMBANDO? 💣
> 
> 1.  **Gripe Aviária no Sul!** 🐔 Imagina o bafafá: descobriram gripe aviária numa granja lá no Rio Grande do Sul! 😱 Santa Catarina já ligou o alerta máximo, e a coisa é séria, viu? A doença pode dar um baita prejuízo e afetar a nossa saúde! 😫
> 
> 2.  **Frango Proibido!** 🚫 E não para por aí: México, Chile e Uruguai, que adoram nosso franguinho, suspenderam a compra! 💸 Imagina o tamanho do rombo no bolso dos produtores! 😩 A China e a União Europeia também entraram na onda e suspenderam a compra da carne de frango brasileira. Mas calma! Comer frango e ovo ainda tá liberado, a doença não passa assim, ok? 😉
> 
> 3.  **Guerra em Gaza!** 💥 Israel resolveu apertar o cerco em Gaza pra ver se o Hamas solta os reféns. 🕊️ Só que a coisa tá feia: foram mais de CEM mortos em um dia! 💔 Gente, que horror! 😥
> 
> 4.  **Mais Mortes em Gaza!** 😢 E não para por aí: mais uma ofensiva de Israel já deixou mais de 146 mortos! 🥺 É gente inocente sofrendo as consequências dessa briga! 😭
> 
> 5.  **Ucrânia em Chamas!** 🔥 A Rússia atacou e matou nove civis na Ucrânia, logo depois de tentarem fazer um acordo! 😠 Parece que a paz está looooonge de acontecer! 😒
> 
> 6.  **Europa com Medo!** 😨 A Europa toda está tão tensa com a Rússia que resolveu comprar sistemas de defesa de Israel! 🛡️ Sinal de que o medo é real e o negócio é se proteger! 💪
> 
> 7.  **Trump Quer Falar com Todo Mundo!** 🗣️ Pasmem! Trump disse que vai botar a boca no trombone com Putin e Zelensky! 💬 Será que vem bomba por aí? 💣
> 
> 8.  **EUA Perdendo a Pose!** 📉 A Moody's tirou a nota máxima de crédito dos EUA! 😲 Depois de mais de um século, a economia americana balançou! 😬 Que babado!
> 
> 9.  **Lava Jato Suspensa!** 🛑 O Brasil parou de ajudar o Peru nas investigações da Lava Jato envolvendo a Odebrecht! 🚧 Dizem que é porque as provas foram consideradas inválidas. 👀
> 
> E aí, o que acharam das fofocas? 🗣️ Não percam os próximos capítulos, porque essa novela está longe de acabar! 😉

--------------------------------------------------------------

✅ Processo concluído.
